In [1]:
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords




In [ ]:
stops =  set(stopwords.words('english')+['com'])
co = CountVectorizer(stop_words=stops)
counts = co.fit_transform(data.Tweet_Text)
pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)

In [ ]:
co = CountVectorizer(ngram_range=(2,2),stop_words=stops)
counts = co.fit_transform(data.Tweet_Text)
pd.DataFrame(counts.sum(axis=0),columns=co.get_feature_names()).T.sort_values(0,ascending=False).head(50)

In [ ]:
words = pd.read_table('https://raw.githubusercontent.com/dwyl/english-words/master/words.txt')

words.columns=['word']
words = words['word'].str.lower().values.tolist()

data['clean_text'] = data.Tweet_Text.apply(lambda x: ' '.join([i for i in x.split(' ') if not (i.startswith('@') or i.startswith('#'))]))
data['clean_text'] = data.clean_text.str.lower().str.replace('[^a-zA]', ' ')

non_list = {}

for sent in tqdm(data.clean_text.str.split().values):
    for token in sent:
        if token not in words:
            non_list[token] = 1 if token not in non_list else non_list[token]+1
            
pd.Series(non_list).sort_values(ascending=False).head(30)

In [ ]:
data.Tweet_Text.str.extractall(r'(\#\w+)')[0].value_counts().head(20).plot.bar()




In [ ]:

vectorizer = CountVectorizer(stop_words=stops)
model = vectorizer.fit(data.clean_text)
docs = vectorizer.transform(data.clean_text)
lda = LatentDirichletAllocation(20)
lda.fit(docs)


In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([(feature_names[i])
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
    
print_top_words(lda,vectorizer.get_feature_names(),10)

In [ ]:
data['topic']=lda.transform(docs).argmax(axis=1)
data.topic.value_counts(normalize=True).plot.bar()

In [ ]:
data[data.topic==0].Tweet_Text.sample(5).values

In [ ]:
data['Date']=pd.to_datetime(data.Date,yearfirst=True)
data.index = data.Date
for i in range(10):
    temp = data[data.topic==i]
    temp.resample('7D').size().plot()
    plt.title('topic %s' %i)
    plt.show()